In [ ]:
from sentinelsat import SentinelAPI

user = 'joanko36'
password = 'JoanKo36'
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

In [ ]:
import geopandas as gpd
import folium
import numpy as np

nReserve = gpd.read_file('NReserve/NaturalReserve_Polygon.shp')

m = folium.Map([44.471092480715676, -0.5780267715454102], zoom_start=14)
folium.GeoJson(nReserve).add_to(m)
m

In [ ]:
from shapely.geometry import MultiPolygon, Polygon

footprint = None
for i in nReserve['geometry']:
    footprint = i

footprint

In [ ]:
products = api.query(footprint,
                    #  date = ('20200727', '20200729'),
                    date = ('20190601', '20190626'),
                    platformname = 'Sentinel-2',
                    # processinglevel = 'Level-2C',
                    cloudcoverpercentage = (0,10)
                    )

In [ ]:
products_gdf = api.to_geodataframe(products)
products_gdf_sorted = products_gdf.sort_values(['cloudcoverpercentage'], ascending=[True])
# products_gdf_sorted = products_gdf_sorted(['title']).startswith('S2C')
# products_gdf_sorted
products_gdf

In [ ]:
# api.download("16e95baf-bbcb-43fe-ab67-5fecea2ffde5")

In [ ]:
import rasterio as rio

# Open Bands 4, 3 and 2 with Rasterio
R10 = 'S2B_MSIL2A_20190605T100039_N0212_R122_T33TTG_20190605T143319.SAFE/GRANULE/L2A_T33TTG_A011731_20190605T100530/IMG_DATA/R10m'
b4 = rio.open(R10+'/T33TTG_20190605T100039_B04_10m.jp2')
b3 = rio.open(R10+'/T33TTG_20190605T100039_B03_10m.jp2')
b2 = rio.open(R10+'/T33TTG_20190605T100039_B02_10m.jp2')

In [ ]:
# Create an RGB image 
# division_factor = np.max(b2.read(1))*255
with rio.open('RGB.tiff','w',driver='Gtiff', width=b4.width, height=b4.height, 
              count=3,crs=b4.crs,transform=b4.transform, dtype=b4.dtypes[0]) as rgb:
    rgb.write(b4.read(1)/(np.max(b4.read(1))*255),1) 
    rgb.write(b3.read(1)/(np.max(b3.read(1))*255),2) 
    rgb.write(b2.read(1)/(np.max(b2.read(1))*255),3) 
    rgb.close()

#read min, max des channels 
# ok

#télécharger niveau 2C (luminance > réflectance) on n'arrive pas à read 2C ? 
# pas de résultats significatifs sur internet

# masque de nuages 
# suffit de filtrer lors de la recherche avant le téléchargement

# peps 

In [ ]:
nReserve_proj = nReserve.to_crs({'init': 'epsg:32633'})
from rasterio.mask import mask
with rio.open("RGB.tiff") as src:
    out_image, out_transform = mask(src, nReserve_proj.geometry,crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
with rio.open("RGB_masked.tif", "w", **out_meta) as dest:
    dest.write(out_image)

In [ ]:
import matplotlib.pyplot as plt
import rasterio.plot as rplt

msk = rio.open(r"RGB_masked.tif")
fig, ax = plt.subplots(1, figsize=(18, 18))
rplt.show(msk.read([1,2,3]))
# rplt.show(msk)
plt.show()

In [ ]:
# b8 = rio.open('T33TTG_20190605T100039_B08_10m.jp2')

# red = b4.read()
# nir = b8.read()

In [ ]:
# ndvi = (nir.astype(float)-red.astype(float))/(nir+red)

# meta = b4.meta

# meta.update(driver='GTiff')
# meta.update(dtype=rio.float32)

# with rio.open('NDVI.tif', 'w', **meta) as dst:
#     dst.write(ndvi.astype(rio.float32))

In [ ]:
# tst = rio.open(r"NDVI.tif")
# fig, ax = plt.subplots(1, figsize=(18, 18))
# # rplt.show(msk.read([1,2,3]))
# rplt.show(tst)
# plt.show()